In [1]:

# Install langgraph and langchain if needed (uncomment when running locally)
# !pip install langgraph>=0.2.0 langchain langchain-openai

from typing import TypedDict, List, Optional, Literal, Dict, Any
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from datetime import datetime


In [2]:

# Demo 1: Simple state without persistence
class SimpleState(TypedDict):
    messages: List[str]
    user_name: str
    counter: int

def process_without_memory(state: SimpleState) -> SimpleState:
    print(f"Current counter: {state['counter']}")
    state['counter'] += 1
    state['messages'].append(f"Processed at count {state['counter']}")
    return state

def demo_state_only():
    print("=== Demo 1: State Only (No Memory) ===\n")
    workflow = StateGraph(SimpleState)
    workflow.add_node("process", process_without_memory)
    workflow.add_edge(START, "process")
    workflow.add_edge("process", END)
    app = workflow.compile()
    # First run
    result1 = app.invoke({"messages": [], "user_name": "Alice", "counter": 0})
    print(f"First run counter: {result1['counter']}, messages: {result1['messages']}\n")
    # Second run resets state
    result2 = app.invoke({"messages": [], "user_name": "Alice", "counter": 0})
    print(f"Second run counter: {result2['counter']}, messages: {result2['messages']}")

# Run Demo 1
demo_state_only()


=== Demo 1: State Only (No Memory) ===

Current counter: 0
First run counter: 1, messages: ['Processed at count 1']

Current counter: 0
Second run counter: 1, messages: ['Processed at count 1']


In [3]:

# Demo 2: Using a checkpointer for persistence

def demo_with_checkpointer():
    print("=== Demo 2: With Checkpointer (Memory) ===\n")
    memory = InMemorySaver()
    workflow = StateGraph(SimpleState)
    workflow.add_node("process", process_without_memory)
    workflow.add_edge(START, "process")
    workflow.add_edge("process", END)
    app = workflow.compile(checkpointer=memory)
    config = {"configurable": {"thread_id": "conversation_123"}}
    # First run
    result1 = app.invoke({"messages": [], "user_name": "Alice", "counter": 0}, config=config)
    print(f"After first run: {result1['counter']}, messages: {result1['messages']}\n")
    # Second run continues from saved state
    saved_state = app.get_state(config).values
    result2 = app.invoke(saved_state, config=config)
    print(f"After second run: {result2['counter']}, messages: {result2['messages']}")

# Run Demo 2
demo_with_checkpointer()


=== Demo 2: With Checkpointer (Memory) ===

Current counter: 0
After first run: 1, messages: ['Processed at count 1']

Current counter: 1
After second run: 2, messages: ['Processed at count 1', 'Processed at count 2']


In [4]:

# Demo 3: Thread Isolation

def demo_thread_isolation():
    print("=== Demo 3: Thread Isolation ===\n")
    memory = InMemorySaver()
    # Define a new state that keeps track of how many times the user has visited support.
    class SupportState(TypedDict):
        messages: List[str]
        user_name: str
        visit: int

    def handle_support(state: SupportState) -> SupportState:
        # Increment the visit count stored in state and append a personalized message
        state['visit'] += 1
        state['messages'].append(f"Support visit {state['visit']} for {state['user_name']}")
        return state

    # Build the workflow graph using the SupportState type
    workflow = StateGraph(SupportState)
    workflow.add_node('support', handle_support)
    workflow.add_edge(START, 'support')
    workflow.add_edge('support', END)
    app = workflow.compile(checkpointer=memory)

    # Simulate the first visit for Alice in her own thread
    config_user1 = {"configurable": {"thread_id": 'alice_support'}}
    state_user1 = {"messages": [], "user_name": 'Alice', "visit": 0}
    result1 = app.invoke(state_user1, config=config_user1)
    print("User 'Alice' first visit:", result1['messages'])

    # Simulate the first visit for Bob in his own thread
    config_user2 = {"configurable": {"thread_id": 'bob_support'}}
    state_user2 = {"messages": [], "user_name": 'Bob', "visit": 0}
    result2 = app.invoke(state_user2, config=config_user2)
    print("User 'Bob' first visit:", result2['messages'])

    # Simulate a second visit for Alice; because we use the same thread_id,
    # her previous state (visit count) is persisted
    saved_alice_state = app.get_state(config_user1).values
    result3 = app.invoke(saved_alice_state, config=config_user1)
    print("User 'Alice' second visit:", result3['messages'])

    # Show that Bob's visit count remains isolated
    saved_bob_state = app.get_state(config_user2).values
    result4 = app.invoke(saved_bob_state, config=config_user2)
    print("User 'Bob' second visit:", result4['messages'])

# Run Demo 3
demo_thread_isolation()


=== Demo 3: Thread Isolation ===

User 'Alice' first visit: ['Support visit 1 for Alice']
User 'Bob' first visit: ['Support visit 1 for Bob']
User 'Alice' second visit: ['Support visit 1 for Alice', 'Support visit 2 for Alice']
User 'Bob' second visit: ['Support visit 1 for Bob', 'Support visit 2 for Bob']


In [6]:
# Demo 4: Config vs State
from langchain_core.runnables import RunnableConfig

class DetailedState(TypedDict):
    messages: List[str]
    current_issue: str
    resolved: bool
    timestamp: str

def process_with_config(state: DetailedState, config: RunnableConfig) -> DetailedState:
    settings = config.get("configurable", {})
    if settings.get("priority") == "high":
        state["messages"].append("HIGH PRIORITY!!!: Escalating immediately!")
    else:
        state["messages"].append("Standard processing...")
    if settings.get("language") == "es":
        state["messages"].append("Hola! ¿Cómo puedo ayudarte?")
    else:
        state["messages"].append("Hello! How can I help you?")
    state["timestamp"] = datetime.now().isoformat()
    return state

def demo_config_vs_state():
    print("=== Demo 4: Config vs State ===\n")
    workflow = StateGraph(DetailedState)
    workflow.add_node("process", process_with_config)
    workflow.add_edge(START, "process")
    workflow.add_edge("process", END)
    app = workflow.compile(checkpointer=InMemorySaver())
    initial_state = {"messages": ["Customer needs help"], "current_issue": "Login problem", "resolved": False, "timestamp": ""}
    config1 = {"configurable": {"thread_id": "test_1", "priority": "normal", "language": "en"}}
    result1 = app.invoke(initial_state.copy(), config=config1)
    print("Normal/English config messages:")
    for msg in result1["messages"][1:]:
        print(" -", msg)
    config2 = {"configurable": {"thread_id": "test_2", "priority": "high", "language": "es"}}
    result2 = app.invoke(initial_state.copy(), config=config2)
    print("High Priority/Spanish config messages:")
    for msg in result2["messages"][1:]:
        print(" -", msg)

# Run Demo 4
demo_config_vs_state()

=== Demo 4: Config vs State ===

Normal/English config messages:
 - Standard processing...
 - Hello! How can I help you?
High Priority/Spanish config messages:
 - Standard processing...
 - Hello! How can I help you?
 - HIGH PRIORITY!!!: Escalating immediately!
 - Hola! ¿Cómo puedo ayudarte?


In [8]:
# Demo 5: Time Travel using checkpoint history


class ConversationState(TypedDict):
    messages: List[str]
    conversation_topic: str
    mood: str

def demo_time_travel():
    print("=== Demo 5: Time Travel with Checkpoints ===\n")
    memory = InMemorySaver()
    
    def conversation_node(state: ConversationState) -> ConversationState:
        """Simulates a conversation response based on topic and mood"""


        topic = state["messages"][-1].split(" ")[-1]
        mood = state["mood"]
        
        if topic == "weather":
            if mood == "happy":
                response = "What a beautiful sunny day! Perfect for a picnic!"
            else:
                response = "Looks like it might rain today. Better stay inside."
        else:
            response = f"Let's talk about {topic}. I'm feeling {mood} today."
            
        state["messages"].append(f"AI: {response}")
        state["conversation_topic"] = topic
        return state
    
    # Build the workflow
    workflow = StateGraph(ConversationState)
    workflow.add_node("chat", conversation_node)
    workflow.add_edge(START, "chat")
    workflow.add_edge("chat", END)
    app = workflow.compile(checkpointer=memory)
    
    # Initial conversation
    config = {"configurable": {"thread_id": "time_travel_conversation"}}
    initial_state = {
        "messages": ["Human: Let's talk about the weather"],
        "mood": "happy"
    }
    
    print("=== Original Timeline ===")
    result1 = app.invoke(initial_state, config=config)
    print(f"Topic: {result1['conversation_topic']}, Mood: {result1['mood']}")
    print(f"Response: {result1['messages'][-1]}")
    

    print(f"\nFull conversation: {result1['messages']}")
    
    # Time travel: Go back to the first checkpoint and change the mood
    print("\n=== Time Travel: Going Back and Changing Mood ===")
    history = list(app.get_state_history(config))


    # The states are returned in reverse chronological order.
    for state in history:
        print(state.next)
        print(state.config["configurable"]["checkpoint_id"])
        print()

    
    if len(history) >= 2:
        # Get the first checkpoint (after initial response)
        first_checkpoint = history[1]  # Second most recent (first response)
        print(f"Going back to checkpoint: {first_checkpoint.config['configurable']['checkpoint_id']}")
        
        # Update the state at that checkpoint - change mood from happy to sad
        new_config = app.update_state(
            first_checkpoint.config,
            {"mood": "sad"},  # This will change how the AI responds
        )

        alt_result = app.invoke(None, config=new_config)
        
        print(f"Modified Timeline - Topic: {alt_result['conversation_topic']}, Mood: {alt_result['mood']}")
        print(f"New Response: {alt_result['messages'][-1]}")
        print(f"\nAlternate conversation: {alt_result['messages']}")


# Run Demo 5
demo_time_travel()

=== Demo 5: Time Travel with Checkpoints ===

=== Original Timeline ===
Topic: weather, Mood: happy
Response: AI: What a beautiful sunny day! Perfect for a picnic!

Full conversation: ["Human: Let's talk about the weather", 'AI: What a beautiful sunny day! Perfect for a picnic!']

=== Time Travel: Going Back and Changing Mood ===
()
1f091ac1-5b4d-6292-8001-771aeccbd29d

('chat',)
1f091ac1-5b4c-6068-8000-dfbe9d890c97

('__start__',)
1f091ac1-5b4a-6844-bfff-f38fcac2cf36

Going back to checkpoint: 1f091ac1-5b4c-6068-8000-dfbe9d890c97
Modified Timeline - Topic: weather, Mood: sad
New Response: AI: Looks like it might rain today. Better stay inside.

Alternate conversation: ["Human: Let's talk about the weather", 'AI: Looks like it might rain today. Better stay inside.']


In [9]:

# Demo 6: Practical memory features
from typing import Dict as DictType, Any as AnyType
class MemoryState(TypedDict):
    messages: List[str]
    user_id: str
    user_memory: DictType[str, AnyType]

def remember_user_preferences(state: MemoryState) -> MemoryState:

    new_preference = state["messages"][-1].split(" ")[-1]

    preferences = state["user_memory"].get("food_preferences", [])
    preferences.append(new_preference)
    state["user_memory"]["food_preferences"] = preferences
    return state


def suggest_a_dessert(state: MemoryState) -> MemoryState:
    preferences = state["user_memory"].get("food_preferences", [])

    if "chocolate" in preferences:
        if "cookies" in preferences:
            state["messages"].append("How about a chocolate cookies for dessert?")
            return state
        elif "cake" in preferences:
            state["messages"].append("How about a chocolate cake for dessert?")
            return state


    state["messages"].append("Can you tell me more about your food preferences?")
    return state

def demo_practical_memory():
    print("=== Demo 6: Practical Memory Features ===\n")
    workflow = StateGraph(MemoryState)
    workflow.add_node("remember", remember_user_preferences)
    workflow.add_node("suggest", suggest_a_dessert)
    workflow.add_edge(START, "remember")
    workflow.add_edge("remember", "suggest")
    workflow.add_edge("suggest", END)
    app = workflow.compile(checkpointer=InMemorySaver())
    config = {"configurable": {"thread_id": "user_123_memory"}}
    # First interaction
    state1 = {
        "messages": ["I like chocolate"],
        "user_id": "user_123",
        "user_memory": {},
    }
    result1 = app.invoke(state1, config=config)
    print("Memory after visit 1:", result1['user_memory'], "\n")
    print("Response after visit 1:", result1['messages'][-1], "\n")

    # Second interaction
    saved = app.get_state(config).values
    saved["messages"].append("I also enjoy cookies")
    result2 = app.invoke(saved, config=config)

    print("Memory after visit 2:", result2['user_memory'], "\n")
    print("Response after visit 2:", result2['messages'][-1], "\n")


# Run Demo 6
demo_practical_memory()


=== Demo 6: Practical Memory Features ===

Memory after visit 1: {'food_preferences': ['chocolate']} 

Response after visit 1: Can you tell me more about your food preferences? 

Memory after visit 2: {'food_preferences': ['chocolate', 'cookies']} 

Response after visit 2: How about a chocolate cookies for dessert? 

